# Brain Networks in Python

BrainNetworksInPython is a tool to perform network analysis over correlation networks of brain regions. 
This tutorial will go through the basic functionality of BrainNetworksInPython, taking us from our inputs (a matrix of structural regional measures over subjects) to a report of local network measures for each brain region, and network level comparisons to a cohort of random graphs of the same degree. 

In [1]:
import numpy as np
import networkx as nx
import BrainNetworksInPython as bnip
import BrainNetworksInPython.datasets as datasets

### Importing data

A BrainNetworksInPython analysis starts with four inputs.
* __regional_measures__
    A pandas DataFrame with subjects as rows. The columns should include structural measures for each brain region, as well as any subject-wise covariates. 
* __names__
    A list of names of the brain regions. This will be used to specify which columns of the __regional_measures__ matrix to want to correlate over.
* __covars__ _(optional)_ 
    A list of your covariates. This will be used to specify which columns of __regional_measure__ you wish to correct for. 
* __centroids__
    A list of tuples representing the cartesian coordinates of brain regions. This list should be in the same order as the list of brain regions to accurately assign coordinates to regions. The coordinates are expected to obey the convention the the x=0 plane is the same plane that separates the left and right hemispheres of the brain. 

In [2]:
# Read in sample data from the NSPN WhitakerVertes PNAS 2016 paper.
df, names, covars, centroids = datasets.NSPN_WhitakerVertes_PNAS2016.import_data()

### Create a correlation matrix
We calculate residuals of the matrix df for the columns of names, correcting for the columns in covars.

In [18]:
df.head()

,Unnamed: 0,nspn_id,occ,centre,study_primary,age_scan,sex,male,age_bin,mri_centre,...,rh_supramarginal_part5,rh_supramarginal_part6,rh_supramarginal_part7,rh_frontalpole_part1,rh_temporalpole_part1,rh_transversetemporal_part1,rh_insula_part1,rh_insula_part2,rh_insula_part3,rh_insula_part4
0,0,10356,0,Cambridge,2K_Cohort,20.761,Female,0.0,4,WBIC,...,2.592,2.841,2.318,2.486,3.526,2.638,3.308,2.583,3.188,3.089
1,1,10702,0,Cambridge,2K_Cohort,16.055,Male,1.0,2,WBIC,...,3.448,3.283,2.740,3.225,4.044,3.040,3.867,2.943,3.478,3.609
2,2,10736,0,Cambridge,2K_Cohort,14.897,Female,0.0,1,WBIC,...,3.526,3.269,3.076,3.133,3.900,2.914,3.894,2.898,3.720,3.580
3,3,10778,0,Cambridge,2K_Cohort,20.022,Female,0.0,4,WBIC,...,2.830,2.917,2.647,2.796,3.401,3.045,3.138,2.739,2.833,3.349
4,4,10794,0,Cambridge,2K_Cohort,14.656,Female,0.0,1,WBIC,...,2.689,3.294,2.820,2.539,2.151,2.734,2.791,2.935,3.538,3.403


In [19]:
names

['lh_lateralorbitofrontal_part2',
 'lh_lateralorbitofrontal_part3',
 'lh_lateralorbitofrontal_part4',
 'lh_lingual_part1',
 'lh_lingual_part2',
 'lh_lingual_part3',
 'lh_lingual_part4',
 'lh_lingual_part5',
 'lh_lingual_part6',
 'lh_medialorbitofrontal_part1',
 'lh_medialorbitofrontal_part2',
 'lh_medialorbitofrontal_part3',
 'lh_middletemporal_part1',
 'lh_middletemporal_part2',
 'lh_middletemporal_part3',
 'lh_middletemporal_part4',
 'lh_middletemporal_part5',
 'lh_parahippocampal_part1',
 'lh_parahippocampal_part2',
 'lh_paracentral_part1',
 'lh_paracentral_part2',
 'lh_paracentral_part3',
 'lh_parsopercularis_part1',
 'lh_parsopercularis_part2',
 'lh_parsopercularis_part3',
 'lh_parsorbitalis_part1',
 'lh_parstriangularis_part1',
 'lh_parstriangularis_part2',
 'lh_pericalcarine_part1',
 'lh_pericalcarine_part2',
 'lh_postcentral_part1',
 'lh_postcentral_part2',
 'lh_postcentral_part3',
 'lh_postcentral_part4',
 'lh_postcentral_part5',
 'lh_postcentral_part6',
 'lh_postcentral_part7

In [3]:
df_res = bnip.create_residuals_df(df, names, covars)

Now we create a correlation matrix over the columns of df_res

In [6]:
M = bnip.create_corrmat(df_res, method='pearson')

## Create a weighted graph

A short sidenote on the BrainNetwork class: This is a very lightweight subclass of the [`Networkx.Graph`](https://networkx.github.io/documentation/stable/reference/classes/graph.html) class. This means that any methods you can use on a `Networkx.Graph` object can also be used on a `BrainNetwork` object, although the reverse is not true. We have added various methods which allow us to keep track of measures that have already been calculated, which, especially later on when one is dealing with 10^3 random graphs, saves a lot of time.  
All BrainNetworksInPython measures are implemented in such a way that they can be used on a regular `Networkx.Graph` object. For example, instead of `G.threshold(10)` you can use `bnip.threshold_graph(G, 10)`.  
Also you can create a `BrainNetwork` from a `Networkx.Graph` `G`, using `bnip.BrainNetwork(network=G)`

Initialise a weighted graph `G` from the correlation matrix `M`. The `parcellation` and `centroids` arguments are used to label nodes with names and coordinates respectively. 

In [8]:
G = bnip.BrainNetwork(network=M, parcellation=names, centroids=centroids)

### Threshold to create a binary graph

We threshold G at cost 10 to create a binary graph with 10% as many edges as the complete graph G. Ordinarily when thresholding one takes the 10% of edges with the highest weight. In our case, because we want the resulting graph to be connected, we calculate a minimum spanning tree first. If you want to omit this step, you can pass the argument `mst=False` to `threshold`.
The threshold method does not edit objects inplace

In [9]:
H = G.threshold(10)

### Calculate nodal summary. 

`calculate_nodal_measures` will compute and record the following nodal measures 

* average_dist (if centroids available)
* total_dist (if centroids available)
* betweenness
* closeness
* clustering coefficient
* degree
* interhem (if centroids are available)
* interhem_proportion (if centroids are available)
* nodal partition
* participation coefficient under partition calculated above
* shortest_path_length

In [ ]:
H.export_nodal_measures.head()

In [12]:
H.calculate_nodal_measures()

        Calculating participation coefficient -           may take a little while


In [16]:
nx.set_node_attributes(H, name="hat", values={x: x**2 for x in H.nodes})

### Report nodal measures as a DataFrame

We can return all nodal attributes in a DataFrame

In [30]:
H.export_nodal_measures().head()

,average_dist,betweenness,centroids,closeness,clustering,degree,hat,interhem,interhem_proportion,module,participation_coefficient,shortest_path_length,total_dist,x,y,z
name,,,,,,,,,,,,,,,,
lh_lateralorbitofrontal_part2,77.5299,0.000914008,"[-56.40355, -40.152663, 1.708876]",0.370474,0.638889,9,0,4,0.444444,0,0.395062,2.68914,697.769,-56.4036,-40.1527,1.70888
lh_lateralorbitofrontal_part3,72.4149,0.0122065,"[-53.140506, -49.843038, 8.264557]",0.492593,0.340986,49,1,19,0.387755,0,0.879633,2.02247,3548.33,-53.1405,-49.843,8.26456
lh_lateralorbitofrontal_part4,69.4417,0.00992147,"[-5.001684, 20.645903, 25.733446]",0.430421,0.387097,31,4,7,0.225806,1,0.62435,2.31461,2152.69,-5.00168,20.6459,25.7334
lh_lingual_part1,73.0082,0.0146519,"[-33.265925, 20.200202, 45.347826]",0.399399,0.388889,9,9,3,0.333333,2,0.395062,2.49438,657.074,-33.2659,20.2002,45.3478
lh_lingual_part2,54.5995,0.00745602,"[-31.958115, 2.146597, 51.26911]",0.331258,0.3,5,16,1,0.2,3,0.36,3.00749,272.998,-31.9581,2.1466,51.2691


### Calculate Global measures

In [25]:
H.calculate_global_measures()

{'assortativity': 0.11824866195197306,
 'average_clustering': 0.4377630334995577,
 'average_shortest_path_length': 2.4554645039565206,
 'efficiency': 0.4703396063727101,
 'modularity': 0.38391579559099454}

In [24]:
H.calculate_rich_club()

{0: 0.09999718397116386,
 1: 0.1044798113763631,
 2: 0.11001836232921477,
 3: 0.11489486744155675,
 4: 0.12263985091944728,
 5: 0.12916106881624123,
 6: 0.1352212389380531,
 7: 0.1392116097998451,
 8: 0.14657683112366876,
 9: 0.1562869997632015,
 10: 0.16321608040201005,
 11: 0.16813186813186815,
 12: 0.16954797779540048,
 13: 0.18207990599294946,
 14: 0.19305341551104263,
 15: 0.20165118679050567,
 16: 0.21093865484109386,
 17: 0.2184748427672956,
 18: 0.22790934555640438,
 19: 0.23662252856883728,
 20: 0.24621212121212122,
 21: 0.25217831813576497,
 22: 0.255760608904181,
 23: 0.271873165002936,
 24: 0.2852903225806452,
 25: 0.288817806210849,
 26: 0.30073880921338547,
 27: 0.3095792578792113,
 28: 0.3200815494393476,
 29: 0.33186813186813185,
 30: 0.35009467704607616,
 31: 0.35879059350503917,
 32: 0.36741519350215,
 33: 0.3756384065372829,
 34: 0.3811217510259918,
 35: 0.3884107860011474,
 36: 0.39567901234567904,
 37: 0.417427701674277,
 38: 0.4382284382284382,
 39: 0.439423076923

## Create a GraphBundle

The `GraphBundle` object is the BrainNetworksInPython way to handle across network comparisons. What is it? Essentially it's a python dictionary with `BrainNetwork` objects as values. 

In [26]:
brain_bundle = bnip.GraphBundle([H], ['NSPN_cost=10'])

In [27]:
brain_bundle

{'NSPN_WhitakerVertes_PNAS2016_cost=10': <BrainNetworksInPython.classes.BrainNetwork at 0x7f877598ab00>}

In [28]:
# Note that 10 is not usually a sufficient number of random graphs to do meaningful analysis,
# it is used here for time considerations
brain_bundle.create_random_graphs('NSPN_cost=10', 10)

        Creating 10 random graphs - may take a little while


In [29]:
brain_bundle

{'NSPN_WhitakerVertes_PNAS2016_cost=10': <BrainNetworksInPython.classes.BrainNetwork at 0x7f877598ab00>,
 1: <BrainNetworksInPython.classes.BrainNetwork at 0x7f87331829b0>,
 2: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8738ca2e80>,
 3: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b480f0>,
 4: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b48ba8>,
 5: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b48ef0>,
 6: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b48198>,
 7: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b48208>,
 8: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8732b481d0>,
 9: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8733377d68>,
 10: <BrainNetworksInPython.classes.BrainNetwork at 0x7f8733377dd8>}

### Report on a GraphBundle

The following method will calculate global measures ( if they have not already been calculated) for all of the graphs in `graph_bundle` and report the results in a DataFrame. We can do the same for rich club coefficients below.

In [12]:
brain_bundle.report_global_measures()

,assortativity,average_clustering,average_shortest_path_length,efficiency,modularity
NSPN_WhitakerVertes_PNAS2016_cost=10,0.118249,0.437763,2.455465,0.470340,0.383916
1,-0.073708,0.225225,2.119850,0.513916,0.000000
2,-0.088696,0.230606,2.116330,0.514315,0.000000
3,-0.109531,0.225835,2.113739,0.514871,0.000000
4,-0.062834,0.226202,2.111430,0.515094,0.000000
5,-0.069585,0.220937,2.126271,0.513408,0.000000
6,-0.065148,0.218086,2.125032,0.513100,0.000000
7,-0.084736,0.233843,2.108755,0.515502,0.000000
8,-0.062070,0.222203,2.118132,0.514172,0.000000
9,-0.057764,0.234477,2.118611,0.513993,0.000000


In [13]:
brain_bundle.report_rich_club()

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
NSPN_WhitakerVertes_PNAS2016_cost=10,0.099997,0.10448,0.110018,0.114895,0.122640,0.129161,0.135221,0.139212,0.146577,0.156287,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.099997,0.10448,0.109894,0.114699,0.122253,0.128452,0.133963,0.137703,0.144447,0.153635,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.099997,0.10448,0.109894,0.114699,0.122253,0.128489,0.134041,0.137785,0.144534,0.153824,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.099997,0.10448,0.109894,0.114699,0.122253,0.128452,0.133963,0.137703,0.144403,0.153587,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.099997,0.10448,0.109894,0.114699,0.122253,0.128452,0.133963,0.137703,0.144577,0.153966,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.099997,0.10448,0.109925,0.114732,0.122323,0.128527,0.134041,0.137866,0.144621,0.153777,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,0.099997,0.10448,0.109894,0.114699,0.122288,0.128527,0.134081,0.137866,0.144664,0.154014,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.099997,0.10448,0.109894,0.114699,0.122253,0.128489,0.134041,0.137785,0.144534,0.153730,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,0.099997,0.10448,0.109894,0.114699,0.122253,0.128489,0.134002,0.137744,0.144447,0.153682,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,0.099997,0.10448,0.109894,0.114699,0.122253,0.128452,0.133963,0.137703,0.144490,0.153777,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
